In [1]:
from pathlib import Path
WORKDIR = Path('/kaggle/input/env-resilience-agent-output/env_resilience_agent')
print("WORKDIR:", WORKDIR)
print("Exists:", WORKDIR.exists())
print("Files:")
for p in sorted(WORKDIR.iterdir()):
    print(" -", p.name)

WORKDIR: /kaggle/input/env-resilience-agent-output/env_resilience_agent
Exists: True
Files:
 - aqi.csv
 - aqi_forecast.csv
 - aqi_history_processed.csv
 - complaints.csv
 - complaints_agg.csv
 - complaints_summary.csv
 - metadata.json
 - ndvi_grid.csv
 - ndvi_overall.csv
 - ndvi_row_summary.csv


In [2]:
import pandas as pd
import json
from pathlib import Path

WORKDIR = Path('/kaggle/input/env-resilience-agent-output/env_resilience_agent')
OUTPATH = Path('/kaggle/working')  # this path is visible in notebook output area

def read_if_exists(name):
    p = WORKDIR / name
    if p.exists():
        try:
            return pd.read_csv(p)
        except Exception as e:
            print(f"Error reading {p}: {e}")
            return None
    else:
        print(f"Missing file: {p}")
        return None

# read the main files
aqi = read_if_exists('aqi.csv')
aqi_history = read_if_exists('aqi_history_processed.csv')
aqi_forecast = read_if_exists('aqi_forecast.csv')
complaints = read_if_exists('complaints.csv')
complaints_agg = read_if_exists('complaints_agg.csv')
complaints_summary = read_if_exists('complaints_summary.csv')
ndvi_grid = read_if_exists('ndvi_grid.csv')
ndvi_overall = read_if_exists('ndvi_overall.csv')
ndvi_low_summary = read_if_exists('ndvi_low_summary.csv')
ndvi_row_summary = read_if_exists('ndvi_row_summary.csv')

# build a compact summary dict (adjust fields as you want)
final_output = {
    "aqi_sample": aqi.head(10).to_dict(orient='records') if aqi is not None else None,
    "aqi_history_rows": len(aqi_history) if aqi_history is not None else None,
    "aqi_forecast": aqi_forecast.to_dict(orient='records') if aqi_forecast is not None else None,
    "complaints_preview": complaints.head(10).to_dict(orient='records') if complaints is not None else None,
    "complaints_agg_rows": len(complaints_agg) if complaints_agg is not None else None,
    "complaints_summary": complaints_summary.to_dict(orient='records') if complaints_summary is not None else None,
    "ndvi_overall": ndvi_overall.to_dict(orient='records') if ndvi_overall is not None else None,
    "ndvi_row_summary": ndvi_row_summary.to_dict(orient='records') if ndvi_row_summary is not None else None,
}

# save JSON to working (so it appears in Output / files area)
OUTPATH.mkdir(parents=True, exist_ok=True)
out_file = OUTPATH / "final_output.json"
with open(out_file, "w", encoding="utf8") as f:
    json.dump(final_output, f, indent=2, ensure_ascii=False)

print("Saved final_output.json to:", out_file)

Missing file: /kaggle/input/env-resilience-agent-output/env_resilience_agent/ndvi_low_summary.csv
Saved final_output.json to: /kaggle/working/final_output.json


In [3]:
from pathlib import Path
WORKDIR = Path('/kaggle/input/env-resilience-agent-output/env_resilience_agent')  # your detected path
print("WORKDIR:", WORKDIR)
print("Files:", [p.name for p in sorted(WORKDIR.iterdir())])

WORKDIR: /kaggle/input/env-resilience-agent-output/env_resilience_agent
Files: ['aqi.csv', 'aqi_forecast.csv', 'aqi_history_processed.csv', 'complaints.csv', 'complaints_agg.csv', 'complaints_summary.csv', 'metadata.json', 'ndvi_grid.csv', 'ndvi_overall.csv', 'ndvi_row_summary.csv']


In [4]:
# AQI agent
import pandas as pd
aqi_history = pd.read_csv(WORKDIR/'aqi_history_processed.csv')
aqi_forecast = pd.read_csv(WORKDIR/'aqi_forecast.csv')

def agent_aqi_forecast():
    return {
        "history_rows": int(len(aqi_history)),
        "history_sample": aqi_history.tail(5).to_dict(orient='records'),
        "forecast": aqi_forecast.to_dict(orient='records')
    }

print("AQI agent ready — history rows:", len(aqi_history))

AQI agent ready — history rows: 30


In [5]:
# NDVI agent
ndvi_grid = pd.read_csv(WORKDIR/'ndvi_grid.csv')
ndvi_overall = pd.read_csv(WORKDIR/'ndvi_overall.csv')
ndvi_row = pd.read_csv(WORKDIR/'ndvi_row_summary.csv')

def agent_ndvi():
    overall = float(ndvi_overall.iloc[0].get('ndvi', ndvi_overall.iloc[0].iat[0])) if not ndvi_overall.empty else None
    return {
        "overall_ndvi": overall,
        "grid_preview": ndvi_grid.head(5).to_dict(orient='records'),
        "row_summary": ndvi_row.to_dict(orient='records')
    }

print("NDVI agent ready — overall:", agent_ndvi()["overall_ndvi"])

NDVI agent ready — overall: 0.3234857586002638


In [6]:
# Complaints agent
complaints = pd.read_csv(WORKDIR/'complaints.csv')
complaints_agg = pd.read_csv(WORKDIR/'complaints_agg.csv')
complaints_summary = pd.read_csv(WORKDIR/'complaints_summary.csv')

def agent_complaints():
    return {
        "sample": complaints.head(5).to_dict(orient='records'),
        "agg": complaints_agg.to_dict(orient='records'),
        "summary": complaints_summary.to_dict(orient='records')
    }

print("Complaints agent ready — sample rows:", len(complaints))

Complaints agent ready — sample rows: 40


In [7]:
# === GEMINI LLM WRAPPER (Using OPENAI_KEY name) ===

import os
from kaggle_secrets import UserSecretsClient

# Load Gemini key (stored under OPENAI_KEY)
us = UserSecretsClient()
os.environ["OPENAI_KEY"] = us.get_secret("OPENAI_KEY")

print("Gemini key found?:", bool(os.environ.get("OPENAI_KEY")))

# Install Gemini client
!pip install -q google-generativeai

import google.generativeai as genai
genai.configure(api_key=os.environ["OPENAI_KEY"])

def ask_llm(prompt):
    try:
        model = genai.GenerativeModel("gemini-pro")
        resp = model.generate_content(prompt)
        return resp.text
    except Exception as e:
        return f"LLM Error: {str(e)}"


# Option A: no-arg wrapper that uses the other agent functions
def agent_llm_summary():
    # get agent outputs (they must be defined earlier)
    aqi_info = agent_aqi_forecast()         # dict
    ndvi_info = agent_ndvi()                # dict
    complaints_info = agent_complaints()    # dict

    # Convert to readable strings for prompt
    aqi_text = f"Forecast date: {aqi_info.get('forecast_date','n/a')}, predicted AQI: {aqi_info.get('pred_aqi','n/a')}"
    ndvi_text = f"Overall NDVI: {ndvi_info.get('overall_ndvi','n/a')}"
    complaints_text = f"Top complaints: {complaints_info.get('top_texts', {})}"

    # build prompt
    prompt = f"""
Summarize the environment status and recommend 3 prioritized actions.

AQI: {aqi_text}
NDVI: {ndvi_text}
Complaints: {complaints_text}

Provide short, prioritized actions (3), and one sentence summary.
"""
    # call your LLM wrapper (ask_llm or llm_client; modify if you use Google Gemini)
    try:
        resp = ask_llm(prompt)   # use your helper that calls OpenAI/Gemini
        # if ask_llm returns a string already, return it; else convert
        return str(resp)
    except Exception as e:
        return f"LLM error: {e}"

Gemini key found?: True
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.9/319.9 kB 11.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
google-cloud-translate 3.12.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.29.5 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
pydrive2 1.21.3 requires cryptography<44, but you have cryptography 46.0.3 which is incompatible.
pydrive2 1.21.3 requires pyOpenSSL<=24.2.1,>=19.1.0, but you have pyopenssl 25.3.0 which is incompatible.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you

In [8]:
import json
from pathlib import Path

final_output = {
    "aqi_agent": agent_aqi_forecast(),
    "ndvi_agent": agent_ndvi(),
    "complaints_agent": agent_complaints(),
    "llm_summary": agent_llm_summary()
}

# Save JSON
out = Path("/kaggle/working/final_output.json")
with open(out, "w", encoding="utf-8") as f:
    json.dump(final_output, f, indent=2, ensure_ascii=False)

print("Saved:", out)
final_output

Saved: /kaggle/working/final_output.json


{'aqi_agent': {'history_rows': 30,
  'history_sample': [{'day': '2025-11-19',
    'aqi_mean': 82.0,
    'aqi_max': 82,
    'pm25_mean': 54.0,
    'ma7': 72.42857142857143},
   {'day': '2025-11-20',
    'aqi_mean': 51.0,
    'aqi_max': 51,
    'pm25_mean': 26.0,
    'ma7': 73.42857142857143},
   {'day': '2025-11-21',
    'aqi_mean': 89.0,
    'aqi_max': 89,
    'pm25_mean': 52.0,
    'ma7': 69.57142857142857},
   {'day': '2025-11-22',
    'aqi_mean': 64.0,
    'aqi_max': 64,
    'pm25_mean': 40.0,
    'ma7': 68.14285714285714},
   {'day': '2025-11-23',
    'aqi_mean': 72.0,
    'aqi_max': 72,
    'pm25_mean': 48.0,
    'ma7': 66.85714285714286}],
  'forecast': [{'date': '2025-11-24', 'pred_aqi': 71},
   {'date': '2025-11-25', 'pred_aqi': 71},
   {'date': '2025-11-26', 'pred_aqi': 71}]},
 'ndvi_agent': {'overall_ndvi': 0.3234857586002638,
  'grid_preview': [{'lat': 26.8, 'lon': 80.9, 'ndvi': 0.231993913668957},
   {'lat': 26.8, 'lon': 80.90689655172415, 'ndvi': 0.084957092657313},
   {'l

In [9]:
!ls -la /kaggle/working | sed -n '1,200p'

total 44
drwxr-xr-x 2 root root  4096 Nov 23 23:33 .
drwxr-xr-x 8 root root  4096 Nov 23 23:33 ..
-rw-r--r-- 1 root root  7377 Nov 23 23:34 final_output.json
---------- 1 root root 24943 Nov 23 23:34 __notebook__.ipynb
